# MUTRITION DATASET AURREPROZESAKETA

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('C:\Users\8orja\OneDrive\Escritorio\3.maila\2.Cuatri\PBL6\Adimen\epi_r.csv')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (Temp/ipykernel_12232/274680970.py, line 4)

In [ ]:
data.head()

Buscar y eliminar datos duplicados.

In [ ]:
dups = data.duplicated()
print('Number of duplicate rows = %d' % (dups.sum()))

print('Number of rows before discarding duplicates = %d' % (data.shape[0]))
data = data.drop_duplicates()
print('Number of rows after discarding duplicates = %d' % (data.shape[0]))

Tratamiento de los NaNs.

In [ ]:
#pd.set_option('display.max_columns', 500)
#pd.set_option("display.max_rows", None)
data.isnull().sum()

Borramos las filas que no tengan calorias.

In [ ]:
data = data[data['calories'].notna()]

In [ ]:
data.isnull().sum()

Al borrar los NaNs de calorias podemos ver que la mayoria de NaNs se han ido. Esto se debe a que no eran recetas.

Ahora, almacenaremos en otra variable los NaNs restantes de proteinas, y veremos que contienen.

In [ ]:
nulls=data[data['protein'].isnull()]
nulls

Viendo el resultado, hemos visto que la mayoría eran recetas que no nos interesan. Como 5 Simple Syropes, que encima tenian 0 calorias. Debido a esto tambien borraremos estos datos.

In [ ]:
data = data[data['protein'].notna()]
data.isnull().sum()

Vemos que ahora qudan unos pocos NaNs en "fat". Así que miraremos su contenido.

In [ ]:
nulls=data[data['fat'].isnull()]
nulls

Analizando el resultado, se observa que la mayoría son cocteles y más sirope. Y muchos de ellos tienen 0 proteinas. Así que tambien los borraremos.

In [ ]:
data = data[data['fat'].notna()]
data.isnull().sum()

Finalmente nos queda un solo NaN en sodio, podemos ver que este tambien tiene 0 en protein y fat, y al ser encima una sola fila, la nborraremos tambien.

In [ ]:
nulls=data[data['sodium'].isnull()]
nulls

In [ ]:
data = data[data['sodium'].notna()]

Una vez limpiado los NaNs vamos a buscar filas que tengan 0 en los atributos fat, sodium, calories y proteinas.

In [ ]:
zeros = data[data["calories"] == 0]
zeros

# Limpieza de datos

Hay 5 filas que no tienen calorias, no tiene sentido que una comida tenga 0 calorías, además son pocas. Borraremos estas 5 filas.

In [ ]:
data = data[data["calories"] != 0]
data.info()

In [ ]:
zerosProtein = data[data["protein"] == 0]
zerosProtein.info()

In [ ]:
zerosFat =  data[data["fat"] == 0]
zerosFat.info()

In [ ]:
zeroAmbos = data[(data['fat'] == 0) & (data['protein'] == 0) ]
zeroAmbos.info()

In [ ]:
zeroSodium = data[data["sodium"]==0]
zeroSodium.info()

Viendo los resultados hemos decidido borrar los que tengan sodio 0 y los que tengan 0 en proteinas y grasa a la vez, ya que estas filas no son recetas que nos interesan.

In [ ]:
data.drop(zeroAmbos.index,axis=0, inplace=True)
#cargamos zerosodium otra vez porque si no da error porque algunos valores ya han sido borrados
zeroSodium = data[data["sodium"]==0]
data.drop(zeroSodium.index,axis=0, inplace=True)

Vamos a borrar las filas que contengan un 1 en las columnas drink y alcoholic.

In [ ]:
bebidas=data[(data['alcoholic'] == 1) | (data['drink'] == 1) | (data['drinks'] == 1) | (data['cake'] == 1)]
bebidas.info()

In [ ]:
data.drop(bebidas.index,axis=0, inplace=True)

# Unificación de columnas

Vamos a comprobar a ver si columnas como cheese y cheedar estan relacionadas.

In [ ]:
cheeseColumns = data[['cheese','cheddar']]
cheeseColumns =  cheeseColumns[cheeseColumns['cheddar']==1]

In [ ]:
cheeseColumns.head()

In [ ]:
fruitColumns = data[['fruit','apple']]
fruitColumns =  fruitColumns[fruitColumns['apple']==1]

In [ ]:
fruitColumns.head()

In [ ]:
meat = data[['meat']]
meat =  meat[meat['meat']==1]
meat.info()

No hay relacion entre las variables genericas y las especificas así que crearemos unas columnas genericas y borraremos las demas.

In [ ]:
def codificador(row, datos):
    columnas = datos.columns

    i = 0
    end = len(columnas)

    for c in columnas:
        if row[c] == 1:
            return 1
        i=i+1
        if i >= end:
            return 0


In [ ]:
#resetear el index, recomendable para iterar
data = data.reset_index()
#creamos un dataframe vacio
dataFinal = pd.DataFrame()
dataFinal = pd.concat((dataFinal,data[['title', 'calories', 'protein', 'fat', 'sodium']]), axis=1)

Empezaremos por unificar las columnas que tienen que ver con si la receta tiene huevos o no, y crearemos una sola columna llamada "huevos" la cual añadiremos al nuevo DataFrame "dataFinal".

In [ ]:
#definir columnas a unificar
columnas = ['egg','egg nog', 'eggplant', 'omelet', 'mayonnaise']

#crear un dataset solo con esas columnas
eggsDatos =  data[columnas]

#columnaEggs = codificador(eggsDatos)
#data['huevos'] = data.apply(columnaEggs, axis=1)
#data = pd.concat((data,columnaEggs), axis=1)

#añadimos al dataFrame final la columna huevos y le asociamos los valores que devuelva la funcion codificador
dataFinal['huevos']=data.apply(lambda row: codificador(row,eggsDatos), axis=1)

In [ ]:
dataFinal.head()

Podemos ver que se ha creado correctamente un dataset unicamente con la columna "huevos" en la cual tiene los valores 1 o 0 dependiendo de si tiene o no huevos la receta.

In [ ]:
#fruta
columnas = ['apple','apple juice','banana','cherry','coconut','fruit','fruit juice','grapefruit','kiwi','cranberry','cranberry sauce',
'macadamia nut','mango','melon','nectarine','orange','papaya','peach','pear','pineapple','plantain','plum','pomegranate','raspberry','strawberry',
'tamarind','tropical fruit','watermelon']
frutasDatos = data[columnas]
dataFinal['frutas'] = data.apply(lambda row: codificador(row,frutasDatos), axis=1)

#mariscos
columnas = ['lobster','mussel', 'octopus', 'oyster', 'salmon', 'sardine', 'seafood', 'shellfish', 'shrimp', 'swordfish', 'trout', 
'tuna', 'caviar','crab','clam', 'fish']
mariscosDatos = data[columnas]
dataFinal['marisco'] = data.apply(lambda row: codificador(row,mariscosDatos), axis=1)

#lacteos
columnas = ['macaroni and cheese','milk/cream', 'mozzarella', 'parmesan', 'ricotta', 'sour cream', 'swiss cheese', 'yogurt',
 'blue cheese', 'buttermilk', 'cheddar', 'cheese', 'cottage cheese','feta','goat cheese', 'cream cheese','custard']
lacteosDatos = data[columnas]
dataFinal['lacteos'] = data.apply(lambda row: codificador(row,lacteosDatos), axis=1)

#frutos secos
columnas = ['nut','nutmeg','peanut','peanut butter','pecan','pistachio','walnut', 'butternut squash','chestnut',
'dried fruit','hazelnut','cashew']
frutossecosDatos = data[columnas]
dataFinal['frutos secos'] = data.apply(lambda row: codificador(row,frutossecosDatos), axis=1)

#verduras
columnas = ['broccoli','broccoli rabe','carrot','celery','leek','mushroom','pea','poblano', 'potato','pumpkin','radicchio',
'radish','root vegetable','shallot','spinach','tomatillo','tomato','turnip']
verdurasDatos = data[columnas]
dataFinal['verduras'] = data.apply(lambda row: codificador(row,verdurasDatos), axis=1)  

#legymbres
columnas = ['chickpea','legume']
legumbresDatos = data[columnas]
dataFinal['legumbres'] = data.apply(lambda row: codificador(row,legumbresDatos), axis=1)

In [ ]:
print(dataFinal)

Guardar los datos limpios en un nuevo csv

In [ ]:
dataFinal.to_csv('Recetas_procesadas.csv', index=False)